In [152]:
import json
from augur.utils import json_to_tree
from Bio import Phylo
tree_file = '../../../../nextstrain-build/phylogenetic_only_camels/auspice/mers.json'
#read in the tree
with open(tree_file, 'r') as f:
    tree_json = json.load(f)

#put tree in Bio.phylo format
tree = json_to_tree(tree_json)

In [153]:
genes = ["Orf3", "Orf4a", "RdRpb", "2Omethyltransferase"]

In [154]:
muts_per_gene_clusters = {}
for node in tree.find_clades():
    muts = node.branch_attrs.get('mutations', {})
    for gene, mutations in muts.items():
        if gene in genes:
            if gene not in muts_per_gene_clusters:
                muts_per_gene_clusters[gene] = {}
            for m in mutations:
                if m not in muts_per_gene_clusters[gene]:
                    muts_per_gene_clusters[gene][m] = 1
                else:
                    muts_per_gene_clusters[gene][m] += 1

print(muts_per_gene_clusters)

{'Orf3': {'E78D': 1, 'C38F': 2, 'L26F': 2, 'S92X': 1, 'V93-': 3, 'N94-': 2, 'L95-': 2, 'F96-': 1, 'D97X': 1, 'Y100X': 1, 'S101-': 1, 'V102-': 1, 'N103X': 1, '*104X': 3, 'S81-': 1, 'T82-': 1, 'H83-': 1, 'D84-': 1, 'G85-': 1, 'P86X': 1, 'T91K': 1, 'V98F': 1, 'G99D': 1, 'H89X': 2, 'X89Y': 1, '-96X': 1, 'X97D': 1, 'X100Y': 1, '-101S': 1, '-102L': 1, 'X103F': 1, 'T82I': 1, 'X89-': 1, 'V90-': 2, 'T91-': 3, 'X92-': 1, '-93X': 1, '-94I': 1, 'X97-': 1, 'V98-': 2, 'G99-': 2, 'X100-': 1, 'S67L': 2, 'P86F': 4, 'F96X': 1, 'D97-': 1, 'Y100-': 1, 'N103D': 2, 'G85D': 3, 'S101X': 1, 'T87N': 2, 'F86-': 2, 'T87-': 1, 'E88-': 2, 'H89-': 1, 'S92-': 2, 'X96-': 1, '-100X': 1, 'S101P': 1, 'G85V': 1, 'T87I': 1, 'S79L': 1, 'I87T': 1, 'L26I': 1, 'L17F': 16, 'A21T': 1, 'V62I': 1, 'S64F': 1, 'P25S': 1, 'V28I': 1, 'T87X': 1, 'X87N': 1, 'X104*': 1, 'D85X': 1, 'X87-': 1, 'V90F': 1, 'L9F': 1, 'S16A': 1, 'S81L': 1, 'T20I': 1, 'R77C': 1, 'G69D': 1, 'L17V': 1, 'F22S': 2, 'S64P': 1, 'D50Y': 1, 'G85T': 1, 'P86L': 8, 'L86F'

In [155]:
sig_muts = {}
premature_stop = {}
mutations_to_count = []
for gene in genes:
    sig_muts[gene] = {}
    premature_stop[gene] = {}
    for m in muts_per_gene_clusters[gene].keys():
        mutations_to_count.append(m)
        if "X" in m:
            premature_stop[gene][m] = muts_per_gene_clusters[gene][m]
        if muts_per_gene_clusters[gene][m] > 3:
            sig_muts[gene][m] = muts_per_gene_clusters[gene][m]

all_muts_by_node = {}
for node in tree.find_clades(): 
    # get the mutations on this node from the 'branch_attrs'
    ### write code here
    muts = node.branch_attrs.get('mutations', {})    
    for gene, mutations in muts.items():
        # Ensure the node and gene are initialized in the dictionary
        if node.name not in all_muts_by_node:
            all_muts_by_node[node.name] = {}
        if gene not in all_muts_by_node[node.name]:
            all_muts_by_node[node.name][gene] = []
        
        # Add mutations to the list for this node and gene
        for m in mutations:
                all_muts_by_node[node.name][gene].append(m)

terminal_count = {}
for node in tree.find_clades(terminal=True):
    path = tree.get_path(node)
    for path_node in path:
        for gene, mutations in all_muts_by_node.get(path_node.name, {}).items():
            if gene in genes:
                if gene not in terminal_count:
                    terminal_count[gene] = {}
                for m in mutations:
                    if m in mutations_to_count:
                        if m not in terminal_count[gene]:
                            terminal_count[gene][m] = 1
                        else:
                            terminal_count[gene][m] += 1

fixed_muts = {}
for gene in genes:
    fixed_muts[gene] = {}
    for m in terminal_count[gene].keys():
        if terminal_count[gene][m] > 50:
            fixed_muts[gene][m] = terminal_count[gene][m]
print("Premature stop codons: " + str(premature_stop))
print("Significant mutations: " + str(sig_muts))
print("Fixed mutations: " + str(fixed_muts))

Premature stop codons: {'Orf3': {'S92X': 1, 'D97X': 1, 'Y100X': 1, 'N103X': 1, '*104X': 3, 'P86X': 1, 'H89X': 2, 'X89Y': 1, '-96X': 1, 'X97D': 1, 'X100Y': 1, 'X103F': 1, 'X89-': 1, 'X92-': 1, '-93X': 1, 'X97-': 1, 'X100-': 1, 'F96X': 1, 'S101X': 1, 'X96-': 1, '-100X': 1, 'T87X': 1, 'X87N': 1, 'X104*': 1, 'D85X': 1, 'X87-': 1}, 'Orf4a': {}, 'RdRpb': {}, '2Omethyltransferase': {}}
Significant mutations: {'Orf3': {'P86F': 4, 'L17F': 16, 'P86L': 8, 'T46I': 8, 'I46T': 8, 'V62F': 6}, 'Orf4a': {'E102Q': 16, 'S106P': 8, 'Q102E': 6}, 'RdRpb': {}, '2Omethyltransferase': {}}
Fixed mutations: {'Orf3': {'S67L': 51, 'L17F': 186, 'P86L': 167, 'T46I': 236, 'I46T': 178}, 'Orf4a': {'E102Q': 211, 'P106S': 303, 'Q102E': 138}, 'RdRpb': {}, '2Omethyltransferase': {}}
